# 6D Pose Estimation - Automated Pipeline

**Instructions**: Click "Runtime" → "Run all" to execute the entire pipeline automatically.

This notebook will:
1. ✅ Setup environment and clone repository
2. ✅ Download and extract LineMOD dataset automatically from Google Drive
3. ✅ Prepare YOLO dataset
4. ✅ Train YOLO detector
5. ✅ Train RGB-only pose model
6. ✅ Evaluate RGB model with metrics
7. ✅ Train RGB-D pose model
8. ✅ Visualize RGB-D results with ground truth
9. ✅ Run final inference demo

**No manual setup required** - the dataset downloads automatically!

## Step 1: Mount Google Drive (Optional for dataset, required for saving models)

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
print("✅ Google Drive mounted successfully")

## Step 2: Clone Repository and Install Dependencies

In [ ]:
!git clone https://github.com/SFR-Vision/6d-pose-estimation.git
%cd 6d-pose-estimation
!pip install -q -r requirements.txt
print("✅ Repository cloned and dependencies installed")

## Step 3: Download and Extract LineMOD Dataset (Automatic)

In [ ]:
!python scripts/setup/setup_data.py

In [ ]:
# Set to True to download pre-trained weights (faster)
# Set to False to train from scratch (recommended for learning)
USE_PRETRAINED = False

if USE_PRETRAINED:
    print("🚀 Using pre-trained weights mode")
    !python scripts/setup/setup_weights.py
else:
    print("🏋️ Training from scratch mode")

## Step 3.5: [OPTIONAL] Download Pre-trained Weights

**Skip training and use pre-trained models instead!**

Set `USE_PRETRAINED = True` below to download pre-trained weights (~3-4 hours of training saved).

If you prefer to train from scratch, keep `USE_PRETRAINED = False`.

## Step 4: Prepare YOLO Dataset

In [ ]:
!python scripts/setup/prepare_yolo.py

## Step 5: Train YOLO Object Detector (15-20 epochs for demo)

In [ ]:
if not USE_PRETRAINED:
    # Train YOLO for 20 epochs (reduce for faster demo)
    !python scripts/training/train_yolo.py --epochs 20
else:
    print("⏭️  Skipping YOLO training (using pre-trained weights)")

## Step 6: Visualize YOLO Detection Results

In [ ]:
!python scripts/visualization/visualize_yolo.py

# Show results
from IPython.display import Image, display
display(Image(filename='runs/detect/linemod_yolo/val_batch0_pred.jpg'))

## Step 7: Train RGB-Only Pose Model (30 epochs for demo)

In [ ]:
if not USE_PRETRAINED:
    # Modify train_rgb.py to train for 30 epochs (faster demo)
    with open('scripts/training/train_rgb.py', 'r') as f:
        content = f.read()
    content = content.replace('EPOCHS = 100', 'EPOCHS = 30')
    with open('scripts/training/train_rgb.py', 'w') as f:
        f.write(content)
    
    !python scripts/training/train_rgb.py
else:
    print("⏭️  Skipping RGB training (using pre-trained weights)")

## Step 8: Evaluate RGB Model with Metrics

In [ ]:
!python scripts/visualization/visualize_final.py

# Show evaluation metrics (printed by script)

## Step 9: Train RGB-D Pose Model (50 epochs for demo)

In [ ]:
if not USE_PRETRAINED:
    # Modify train_rgbd.py to train for 50 epochs
    with open('scripts/training/train_rgbd.py', 'r') as f:
        content = f.read()
    content = content.replace('EPOCHS = 150', 'EPOCHS = 50')
    with open('scripts/training/train_rgbd.py', 'w') as f:
        f.write(content)
    
    !python scripts/training/train_rgbd.py
else:
    print("⏭️  Skipping RGB-D training (using pre-trained weights)")

## Step 10: Visualize RGB-D Results (Ground Truth vs Prediction)

In [ ]:
!python scripts/visualization/visualize_rgbd.py

## Step 11: Run Final Inference Demo

In [ ]:
!python scripts/inference/inference_rgbd.py

## Step 12: Save Models Back to Google Drive

In [ ]:
import shutil
import os

# Create backup directory
backup_dir = "/content/drive/MyDrive/LineMOD/trained_models_backup"
os.makedirs(backup_dir, exist_ok=True)

# Save all trained models
shutil.copytree("weights", f"{backup_dir}/weights", dirs_exist_ok=True)
shutil.copytree("weights_rgbd", f"{backup_dir}/weights_rgbd", dirs_exist_ok=True)
shutil.copytree("runs", f"{backup_dir}/runs", dirs_exist_ok=True)

print("✅ All models saved to Google Drive!")
print(f"Location: {backup_dir}")

# Print final summary
print("\n" + "="*50)
print("PIPELINE COMPLETE!")
print("="*50)
print("✅ YOLO detector trained")
print("✅ RGB pose model trained")
print("✅ RGB-D pose model trained")
print("✅ All models backed up to Google Drive")
print("\nCheck the visualizations above to see results!")